## Thư viện

In [1]:
import sklearn
import pandas
import numpy as np

## Load tập dữ liệu doanh số bán nhà

Tập dữ liệu từ doanh số bán nhà quận King, Seatle, WA.

In [2]:
full_data = pandas.read_csv("kc_house_data.csv", index_col=0)

## Tạo các đặc trung mới

Như ở lab 2 (*Lab-2.ipynb*), chúng ta sẽ xem xét các đặc trưng có các biến đổi đầu vào.

In [3]:
from math import log, sqrt
full_data['sqft_living_sqrt'] = full_data['sqft_living'].map(sqrt)
full_data['sqft_lot_sqrt'] = full_data['sqft_lot'].map(sqrt)
full_data['bedrooms_square'] = full_data['bedrooms'] ** 2

# Trong tập dữ liệu, 'floors' được xác định là type string, 
# nên chúng ta sẽ chuyển chúng thành float trước khi tạo đặc trưng mới. 
full_data['floors'] = full_data['floors'].astype(float) 
full_data['floors_square'] = full_data['floors'] ** 2

* Bình phương bedrooms sẽ tăng phân tách giữa ít phòng ngủ (chẳng hạn 1) và nhiều phòng ngủ (chẳng hạn 4) vì 1^2 = 1 còn 4^2 = 16. Do đó, biến này sẽ ảnh hưởng lớn tới các ngôi nhà có nhiều phòng ngủ.
* Mặt khác, căn bậc hai của sqft_living sẽ giảm phân tách giữa nhà lớn và nhà nhỏ. Chủ ngôi nhà cũng sẽ không vui hơn nếu nhà rộng gấp đôi.

# Tìm hiểu trọng số hồi quy với L1 penalty

Hãy khớp mô hình với tất cả đặc trưng hiện có cộng với các đặc trưng vừa tạo.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Áp dụng L1 penalty cần thêm tham số (`alpha=l1_penalty`) cho mô hình  `Lasso` của Sklearn. (Các công cụ khác cũng phân tách các triển khai của LASSO). Khá giống Hồi quy Ridge/L2, các đặc trưng cũng cần được co giãn để đảm bảo đồng đều ở giữa.

In [5]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
l1_penalty=5e4
full_features = scaler.fit_transform(full_data[all_features].values)
full_labels = full_data['price'].values
model = Lasso(alpha=l1_penalty).fit(full_features, full_labels)

Tìm các đặc trưng có trọng số khác 0.

In [6]:
# Bạn có biết numpy cũng có boolean selector tích hợp sẵn?
print(model.coef_)
print(all_features[3],all_features[9],all_features[10],all_features[12],all_features[15])

[     0.              0.              0.         132571.09360631
      0.             -0.             -0.              0.
      0.          14623.33961421  29004.06421249      0.
  90207.54789031      0.              0.         -10722.34912003
      0.        ]
sqft_living waterfront view grade yr_built


Lưu ý rằng phần lớn trọng số được đặt thành 0. Vì vậy, bằng cách đặt L1 penalty đủ lớn, chúng ta có thể thực hiện lựa chọn tập con.

***QUIZ***:
Theo list các trọng số này, những đặc trưng nào đã được chọn? 
  **sqft_living , waterfront , view , grade , yr_built**

# Lựa chọn L1 penalty

Để tìm một L1 penalty tốt, chúng ta sẽ khám phá nhiều giá trị sử dụng tập kiểm định. Hãy chia dữ liệu thành tập huấn luyện, tập kiểm định và tập kiểm tra:
* Chia dữ liệu bán hàng thành 2 tập: tập huấn luyện và tập kiểm tra (9/1)
* Chia tiếp tập huấn luyện thành 2 tập: tập huấn luyện và kiểm định (5/5)

Hãy dùng seed = 1 để có cùng kết quả!

In [7]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(full_data,train_size = 0.9, random_state=0)
huanluyen_data,kiemdinh_data = train_test_split(full_data,train_size = 0.5, random_state=0)

# Cookie cho những ai cần copy cell 

Tiếp theo, chúng ta sẽ viết một vòng lặp như sau: 
* Với `l1_penalty` trong phạm vi 21 bước giữa [1, 10^9] (sử dụng `np.logspace(0, 9, num=21)`.)
    * Khớp mô hình hồi quy với `l1_penalty` trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty` trong tham số.
    * Tính RSS trên dữ liệu KIỂM ĐỊNH (sử dụng `.predict()`) cho `l1_penalty`
* Báo lại `l1_penalty` nào cho RSS thấp nhất trong dữ liệu KIỂM ĐỊNH.

In [8]:
# Mức độ trợ giúp: Bình thường.
l1_penalty = np.logspace(0, 9, num=21)
RSS = []
features1 = StandardScaler().fit_transform(huanluyen_data[all_features].values)
features2 = StandardScaler().fit_transform(kiemdinh_data[all_features].values)
for penalty in l1_penalty:
    model = Lasso(alpha=penalty).fit(features1,huanluyen_data['price'])
    y_pred = model.predict(features2)
    rss = np.sum((kiemdinh_data['price'] - y_pred )**2)
    RSS.append(rss)
print(RSS)  

c:\users\v\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.649e+13, tolerance: 1.421e+11
  model = cd_fast.enet_coordinate_descent(
c:\users\v\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.650e+13, tolerance: 1.421e+11
  model = cd_fast.enet_coordinate_descent(
c:\users\v\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the featu

[477137332129531.06, 477148567250585.56, 477180360831126.94, 477270989672931.06, 477534537324713.1, 478341778404632.6, 481129223794446.5, 490330448369652.5, 534241235980652.9, 553919784978954.5, 597497417794524.6, 786006567086542.8, 1477650369721896.5, 1491581209042466.0, 1491581209042466.0, 1491581209042466.0, 1491581209042466.0, 1491581209042466.0, 1491581209042466.0, 1491581209042466.0, 1491581209042466.0]


In [9]:
print(l1_penalty[RSS.index(min(RSS))])
min(RSS)

1.0


477137332129531.06

***QUIZ:*** Giá trị tốt nhất cho `l1_penalty` là bao nhiêu?  **1.0**

In [10]:
# Qua quan sát hay tính toán?
features1 = StandardScaler().fit_transform(huanluyen_data[all_features].values)
features2 = StandardScaler().fit_transform(kiemdinh_data[all_features].values)
model = Lasso(alpha=l1_penalty[RSS.index(min(RSS))]).fit(features1,huanluyen_data['price'])
print(len(model.coef_[model.coef_!=0]))

17


c:\users\v\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.649e+13, tolerance: 1.421e+11
  model = cd_fast.enet_coordinate_descent(


***QUIZ***
Với giá trị L1 penalty này, chúng ta có bao nhiêu trọng số khác 0? **17**

In [11]:
# Thú vị phải không?

# Limit the number of nonzero weights Giới hạn số trọng số khác 0

Sẽ ra sao nếu chúng ta muốn giới hạn, 5 đặc trưng chẳng hạn? Điều này quan trọng nếu chúng ta muốn suy ra "quy tắc ngón tay cái" --- mô hình có thể diễn giải chỉ với một vài đặc trưng.

Trong phần này, chúng ta sẽ triển khai quy trình đơn giản gồm 2 giai đoạn :
1. Thăm dò phạm vi lớn các giá trị `l1_penalty` để tìm vùng các giá trị `l1_penalty` hẹp hơn mà mô hình chắc chắn sẽ có số lượng trọng số khác 0 mong muốn.
2. Thăm dò tiếp vùng hẹp đã thấy để tìm gái trị tốt cho `l1_penalty` đạt được độ thưa thớt mong muốn. Ở đây chúng ta sử dụng tập kiểm định để chọn giá trị tốt nhất cho `l1_penalty`. 

In [12]:
max_nonzeros = 5

## Khám phá phạm vi giá trị lớn hơn để tìm phạm vi hẹp với độ thưa thớt mong muốn

Hãy xác định một loạt các `l1_penalty_values` có thể:

In [13]:
l1_penalty_values = np.logspace(3, 5, num=21)

Giờ hãy triển khi vòng lặp tìm kiếm trong không gian có các giá trị `l1_penalty` có thể:

* Với `l1_penalty` trong `np.logspace(3, 5, num=21)`:
    * Khớp mô hình hồi quy với `l1_penalty` đã biết trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty` trong tham số.
    * Trích xuất trọng số của mô hình và đếm số trọng số khác 0. Lưu con số này vào một list. 
        * Gợi ý: `model.coef_` cho các tham số/hệ số đã tìm thấy (intercept) ở dạng mảng numpy. Sau đó có thể dùng array\[condition\] cho list các giá trị truyền điều kiện, hoặc chỉ dùng `np.count_nonzero()` có sẵn.

In [14]:
# Hoặc có thể thực hiện với python thuần túy. Nó không ảnh hưởng đáng kể tới chất lượng. 
W = []
sll = []
features1 = StandardScaler().fit_transform(huanluyen_data[all_features].values)
features2 = StandardScaler().fit_transform(kiemdinh_data[all_features].values)
for penalty in l1_penalty_values:
    model = Lasso(alpha=penalty).fit(features1,huanluyen_data['price'])
    W.append(list(model.coef_))
    sll.append(len(model.coef_[model.coef_!=0]))
print(sll)

[14, 14, 14, 14, 14, 14, 12, 10, 10, 10, 10, 9, 6, 5, 5, 5, 5, 5, 4, 4, 3]


Trong phạm vi lớn này, chúng ta có thể tìm 2 đầu phạm vi hẹp mong muốn của `l1_penalty`. Ở một đầu, các giá trị `l1_penalty` có quá ít giá trị khác 0, còn đầu kia lại có quá nhiều giá trị khác 0.

Hãy tìm:
* `l1_penalty` nhỏ nhất có các số khác 0 bằng `max_nonzeros` (nếu chọn penalty nhỏ hơn giá trị này chắc chắn sẽ có rất nhiều trọng số khác 0).
    * Lưu giá trị này trong biến `l1_penalty_min` (sẽ sử dụng nó sau).
* `l1_penalty` lớn nhất có các số khác 0 bằng `max_nonzeros` (nếu chọn penalty lớn hơn giá trị này chắc chắn sẽ có rất ít trọng số khác 0).
    * Lưu giá trị này trong biến `l1_penalty_max` (sẽ sử dụng nó sau).


*Gợi ý: có nhiều cách để thực hiện, chẳng hạn:*
* Lập trình trong vòng lặp trên.
* Tạo một list với số lượng khác 0 cho từng giá trị `l1_penalty` và kiểm tra nó để tìm ranh giới thích hợp.

In [15]:
l1_penalty_min = min(np.array(l1_penalty_values)[np.where(np.array(sll) == max_nonzeros)[0]]) #
l1_penalty_max = max(np.array(l1_penalty_values)[np.where(np.array(sll) == max_nonzeros)[0]]) #
print(l1_penalty_min,l1_penalty_max)

19952.62314968879 50118.72336272725


***QUIZ.*** Chúng ta tìm thấy các giá trị nào lần lượt cho `l1_penalty_min` và `l1_penalty_max`? **19952.62314968879 50118.72336272725**

## Khám phá phạm vi nhỏ các giá trị để tìm giải pháp với đúng số lượng các số khác 0 có RSS trong tập kiểm định nhỏ nhất

Chúng ta sẽ khám phá vùng hẹp các giá trị `l1_penalty` đã tìm thấy:

In [16]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* Với `l1_penalty` trong `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Khớp mô hình hồi quy với `l1_penalty` đã biết trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty`.
    * Đo lường RSS của mô hình đã tìm hiểu trong tập KIỂM ĐỊNH.

Tìm mô hình có RSS nhỏ nhất trong tập KIỂM ĐỊNH và độ thưa thớt *bằng*  `max_nonzeros`.

In [17]:
# Xem quiz bên dưới
penal = 0
RSS = float("inf")
W = []
for penalty in l1_penalty_values:
    model = Lasso(alpha=penalty).fit(features1,huanluyen_data['price'])
    y_pred = model.predict(features2)
    rss = np.sum((kiemdinh_data['price'] - y_pred )**2)
    if rss < RSS:
        RSS = rss
        penal = penalty
        W = model.coef_
print(RSS)
print(penal)
print(np.array(all_features)[np.where(np.array(W) != 0)[0]]) 

570273645842013.5
19952.62314968879
['sqft_living' 'waterfront' 'view' 'grade' 'yr_built']


***QUIZ***
1. Giá trị của `l1_penalty` trong phạm vi hẹp hơn có RSS thấp nhất trong tập KIỂM ĐỊNH và độ thưa *bằng* `max_nonzeros` là? **19952.62314968879**
2. Các đặc trung nào trong mô hình này có các hệ số khác 0? **'sqft_living' 'waterfront' 'view' 'grade' 'yr_built'**